In [1]:
%matplotlib inline 
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils import data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# データ
train_data = pd.read_csv("./dataset/NKI_RS_train_data.csv",index_col=0)
train_label = pd.read_csv("./dataset/NKI_RS_train_label.csv",index_col = 0)
test_data = pd.read_csv("./dataset/NKI_RS_test_data.csv", index_col = 0)
train_label = train_label["sex"]

In [3]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits = 5, shuffle=True, random_state = 0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 訓練用の関数
from train_eval_loop import train_valid_loop
# データをtensor型にかえる関数
from datapreparation import df2tensor

In [4]:
# modelのimport
from model_NN_exp import Net_deeper
# 訓練/検証用関数
from train_eval_loop_test import train_valid_loop

In [5]:
num = 0
n_epoch = 500
for train_idx, valid_idx in skf.split(train_data, train_label):
    print(f"--------------------This is {num + 1}th fold.10epochごとの結果を以下に出力--------------------")
    # modelの初期化
    model = Net_deeper().to(device)
    optimizer = optim.SGD(model.parameters(), lr = 1e-2, momentum = 0.9, weight_decay = 5e-3)
    best_score = 0
    # 分割した訓練データと検証データをtensor型に変換
    trainloader, validloader, valid_data, valid_label = df2tensor(
        train_data.iloc[train_idx], train_label.iloc[train_idx], train_data.iloc[valid_idx], train_label.iloc[valid_idx], batch_size = 64)
    # モデルの訓練と評価, earlystoppipngによっていちばんauc_scoreの高かった
    train_acc_list, train_loss_list, valid_acc_list, valid_loss_list, auc_score_list = train_valid_loop(
        trainloader, validloader, valid_data, valid_label, model, n_epoch, optimizer)
    auc_score = np.max(auc_score_list)
    if auc_score > best_score:
        best_score = auc_score
        # この辺のfoldごとの最良モデルの読み込みをもう少しうまく、簡潔に書けないか
        best_model = model.load_state_dict(torch.load("model.pth"))
    for a,b,c,d,e in zip(
        train_loss_list, train_acc_list,
        valid_loss_list, valid_acc_list, auc_score_list):
        print(f"train loss:{a:.5f},train acc:{b:.5f}, valid loss:{c:.5f}, valid acc:{d:.5f}, auc_score:{e:.5f}")
    num += 1


This is 1th fold.10epochごとの結果を以下に出力--------------------
Early Stopping is working in 12th epoch.
best auc score in validation is 0.7393176020408163
train loss:0.66342,train acc:0.62346, valid loss:0.69174, valid acc:0.59259, auc_score:0.66486
train loss:0.31667,train acc:0.99846, valid loss:0.62980, valid acc:0.66049, auc_score:0.72385
This is 2th fold.10epochごとの結果を以下に出力--------------------
Early Stopping is working in 13th epoch.
best auc score in validation is 0.7523915816326531
train loss:0.66587,train acc:0.58025, valid loss:0.69733, valid acc:0.39506, auc_score:0.71301
train loss:0.31524,train acc:1.00000, valid loss:0.60815, valid acc:0.67901, auc_score:0.70265
This is 3th fold.10epochごとの結果を以下に出力--------------------
Early Stopping is working in 19th epoch.
best auc score in validation is 0.767059948979592
train loss:0.67346,train acc:0.59877, valid loss:0.70292, valid acc:0.39506, auc_score:0.58642
train loss:0.31642,train acc:0.99846, valid loss:0.60319, valid acc:0.69136, auc_s

In [8]:
print(best_model)

<All keys matched successfully>


In [11]:
model = Net_deeper().to(device)
model.load_state_dict(torch.load("model.pth"))
test_data_tensor = torch.tensor(np.array(test_data.astype('f')))
test_prediction = model.forward(test_data_tensor.to(device))
# print(bn_prediction)
print(test_prediction[:,1])

tensor([2.0049e-02, 9.4324e-01, 9.8817e-01, 4.7000e-03, 9.3661e-01, 4.0241e-01,
        9.4947e-01, 3.6128e-01, 9.9902e-01, 9.8239e-01, 9.9768e-01, 9.9173e-01,
        9.8982e-01, 9.7014e-01, 4.9089e-01, 5.2840e-01, 8.7702e-01, 8.2606e-01,
        3.5409e-01, 9.8627e-01, 9.7744e-01, 7.5345e-03, 2.0753e-02, 2.8691e-01,
        7.5752e-01, 2.3509e-03, 8.8028e-01, 2.6827e-01, 3.6327e-02, 8.5916e-01,
        1.0939e-01, 1.8605e-01, 3.4798e-02, 5.0400e-01, 1.7239e-07, 6.5795e-01,
        4.6317e-02, 9.9331e-01, 8.3681e-01, 5.5799e-04, 9.9862e-01, 1.4616e-01,
        2.4258e-01, 2.6921e-01, 9.4909e-01, 2.6443e-04, 9.9768e-01, 9.9667e-01,
        9.9571e-01, 9.9835e-01, 6.3754e-01, 4.6275e-02, 5.2967e-01, 5.8342e-01,
        9.2873e-01, 1.9283e-03, 9.5859e-01, 9.7542e-01, 3.0201e-03, 9.6249e-01,
        2.1290e-01, 2.4673e-02, 2.4967e-05, 3.5775e-01, 1.0610e-01, 5.0361e-03,
        9.8631e-01, 9.9906e-01, 5.1176e-02, 9.9863e-01, 1.0989e-02, 9.2207e-01,
        2.0827e-01, 9.8395e-01, 7.6182e-

In [14]:
df_submission = pd.read_csv("./dataset/sample_submission.csv",header = 0)
df_submission['sex'] = test_prediction[:, 1].to('cpu').detach().numpy().copy()
df_submission

,subject_id,sex
0,A00062361,0.020049
1,A00039143,0.943235
2,A00031605,0.988168
3,A00037848,0.004700
4,A00060264,0.936609
...,...,...
76,A00039075,0.038971
77,A00037768,0.994371
78,A00058621,0.967057
79,A00033903,0.992486


In [16]:
df_submission.to_csv("./submission_1st/DNN_test.csv")